In [1]:
!pip install -U bitsandbytes chromadb langchain-community

In [1]:
from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig
import torch
import re
from tqdm import tqdm
import json
import pandas as pd
import numpy as np

import os
from pathlib import Path
from typing import List, Dict
from datetime import datetime
from langchain.schema import Document
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.vectorstores import Chroma
import chromadb

In [2]:
SYSTEM_PROMPT = (
    "Ты виртуальный ассистент. Твоя задача - помогать людям с поэзией, отвечая на их запросы."
)

USER_PROMPT_NER = '''
Ты — помощник по анализу русской поэзии.
Из запроса пользователя выдели имена авторов и названия стихотворений, если они явно упоминаются.
Ответ верни строго в JSON-формате с двумя полями:
- "authors": список ФИО авторов (только фамилия и имя, без отчества), если они явно упоминаются в запросе
- "poems": список названий стихотворений, если они явно упоминаются в запросе. Если в запросе нет явных названий стихотворений, ничего не пиши!

Не пиши ничего, кроме JSON. Если ничего не найдено — верни пустые списки.
**Не добавляй ни одного автора или названия по смыслу, теме или ассоциации. Только то, что явно названо.**

Пример стуктуры ответа:
{{
  "authors": ["Александр Пушкин", "Михаил Лермонтов"],
  "poems": ["К Чаадаеву", "Бородино"]
}}

Запрос: {query}
Ответ:
'''

USER_PROMPT_REWRITING = '''
Ты — помощник по тематике русской поэзии.
На основе запроса пользователя сформулируй ключевые образы и ассоциации, связанные с его смыслом. Это могут быть чувства, образы природы, исторические мотивы и т.п.
Ответ верни строго в JSON-формате с одним полем:
- "keywords": список ключевых слов или фраз, которые помогут найти релевантные стихи.

**Не пиши ничего, кроме JSON. Если образы не подобрать — верни пустой список.**

Пример:
{{
  "keywords": ["осень", "листья", "тоска", "разлука"]
}}

Запрос: {query}
Ответ:
'''

USER_PROMPT_MAIN = '''
Ты — виртуальный ассистент, помогающий людям находить и интерпретировать русскую поэзию.
Ниже приведены отрывки стихотворений, которые могут быть релевантны запросу пользователя. Используй их как источник для вдохновения или цитирования в ответе.
Не выдумывай новых стихов и не приписывай авторам лишнего — работай только с тем, что дано в контексте.
Будь образным, но точным. Если пользователь просит «подборку» — выбери релевантные строки. Если просит «написать стихи» — сочини, вдохновляясь контекстом.

Контекст:
{context}

Запрос пользователя: {query}
Ответ:
'''

MODEL_NAME = "t-tech/T-lite-it-1.0"
EMBED_MODEL_NAME = "sergeyzh/BERTA"

DATA_PATH = 'russianPoetryWithTheme.csv'
AUTHORS_COL = 'author'
POEMS_COL = 'name'
TXT_COL = 'text'

RAG_METADATA_COLS = ['date_to', 'author', 'name']
RAG_TXT_COL = 'text'
RAG_SEARCH_METHOD = 'marginal'

In [3]:
class Preprocessor:
    def __init__(self, model, tokenizer, data, authors_col, poems_col):
        self.tokenizer = tokenizer
        self.model = model
        self.data_ini = data.copy()
        self.authors_col = authors_col
        self.poems_col = poems_col

        self.data_ini['norm_author'] = self.data_ini[self.authors_col].apply(lambda x: x if pd.isnull(x) else self.normalize_author(x))
        self.data_ini['norm_poem'] = self.data_ini[self.poems_col ].apply(lambda x: x if pd.isnull(x) else self.normalize_poem(x))


    def _structout_json_parser_ner(self, txt):
      txt_clean = txt.replace("`", "").replace("json", "").strip()
      try:
        res_dct = json.loads(txt_clean)
        authors = res_dct.get('authors', [])
        poems = res_dct.get('poems', [])
      except json.JSONDecodeError:
        print('JSONDecodeError from response')
        authors = []
        poems = []

      return authors, poems

    def _structout_json_parser_rewrite(self, txt):
      txt_clean = txt.replace("`", "").replace("json", "").strip()
      try:
        res_dct = json.loads(txt_clean)
        keywords = res_dct.get('keywords', [])
      except json.JSONDecodeError:
        print('JSONDecodeError from response')
        keywords = []

      return keywords

    def normalize_author(self, name):
        return name.replace('ё', 'е').lower().strip()

    def normalize_poem(self, name):
        return re.sub(r'[^а-яА-Яa-zA-Z0-9 ]', '', name).lower().strip()

    def _check_data_contains(self, author=None, poem=None):
      if author:
        author_norm = self.normalize_author(author)
      if poem:
        poem_norm = self.normalize_poem(poem)

      if author and poem:
        match_df = self.data_ini.loc[(self.data_ini.norm_author == author_norm) & (self.data_ini.norm_poem == poem_norm)]
      elif author and not poem:
        match_df = self.data_ini.loc[(self.data_ini.norm_author == author_norm)]
      else:
        match_df = self.data_ini.loc[(self.data_ini.norm_poem == poem_norm)]

      return match_df.shape[0] > 0

    def _rag_router(self, authors, poems):
      res_dct = {}

      if not authors and not poems:
        res_dct['is_direct'] = 0
        return res_dct

      if authors and not poems: # есть авторы, нет стихов
        res_dct['authors'] = res_dct.get('authors', []) + [author for author in authors if self._check_data_contains(author=author)]
        res_dct['is_direct'] = 0
      elif not authors and poems: # нет авторов, есть стихи
        res_dct['poems'] = res_dct.get('poems', []) + [poem for poem in poems if self._check_data_contains(poem=poem)]
        res_dct['is_direct'] = 0
      elif len(authors) > len(poems): # авторов больше, чем стихов - считаем ошибкой распознавания
        res_dct['authors'] = res_dct.get('authors', []) + [author for author in authors if self._check_data_contains(author=author)]
        res_dct['is_direct'] = 0
      elif len(authors) == 1 and poems: # 1 автор, много стихов
        poems_checked_1author = [poem for poem in poems if self._check_data_contains(author=authors[0], poem=poem)]
        if len(poems_checked_1author) > 0:
          res_dct['poems'] = poems_checked_1author
          res_dct['authors'] = [authors[0]]*len(poems_checked_1author)
          res_dct['is_direct'] = 1
        else:
          res_dct['authors'] = [authors[0]]
          res_dct['is_direct'] = 0
      elif len(poems) > len(authors): # стихов больше, чем авторов, но автор не 1 - считаем ошибкой распознавания
        res_dct['authors'] = res_dct.get('authors', []) + [author for author in authors if self._check_data_contains(author=author)]
        res_dct['is_direct'] = 0
      else:
        ap_checked = [(author, poem) for author, poem in zip(authors, poems) if self._check_data_contains(author=author, poem=poem)]
        res_dct['authors'] = [i[0] for i in ap_checked]
        res_dct['poems'] = [i[1] for i in ap_checked]
        res_dct['is_direct'] = 1

      return res_dct

    def process_response_ner(self, response):
      splitted = response.split('assistant')
      if len(splitted) > 1:
        splitted = splitted[1]
      else:
        splitted = splitted[-1]

      authors, poems = self._structout_json_parser_ner(splitted)

      # когда модель галлюционирует (без стиха с запросе), то пишет много вариантов на тему
      # + более 2 стихов скорее всего людям будет лень и в целом странно в одном запросе
      if len(poems)/max(len(authors), 1) > 2:
        poems = []

      res_ner = self._rag_router(authors, poems)

      return res_ner

    def process_response_rewrite(self, response):
      splitted = response.split('assistant')
      if len(splitted) > 1:
        splitted = splitted[1]
      else:
        splitted = splitted[-1]

      keywords = self._structout_json_parser_rewrite(splitted)

      return keywords

    def generate(self, query, system_prompt, user_prompt, max_new_tokens=250, temperature=0.7, top_p=0.9):
        uprompt = user_prompt.format(query=query)
        messages = [
            {"role": "system", "content": system_prompt},
            {"role": "user", "content": uprompt}
        ]
        input_ids = self.tokenizer.apply_chat_template(messages, return_tensors="pt").to(self.model.device)

        with torch.no_grad():
            output = self.model.generate(
                input_ids,
                max_new_tokens=max_new_tokens,
                do_sample=True,
                temperature=temperature,
                top_p=top_p,
                pad_token_id=self.tokenizer.eos_token_id
            )
        generated = self.tokenizer.decode(output[0], skip_special_tokens=True)

        return generated

    def get_query_ner(self, query, system_prompt, user_prompt, max_new_tokens=250, temperature=0.7, top_p=0.9):
      generated = self.generate(query, system_prompt, user_prompt, max_new_tokens, temperature, top_p)
      result = self.process_response_ner(generated)
      return result

    def get_query_rewrite(self, query, system_prompt, user_prompt, max_new_tokens=250, temperature=0.7, top_p=0.9):
      generated = self.generate(query, system_prompt, user_prompt, max_new_tokens, temperature, top_p)
      result = self.process_response_rewrite(generated)
      return result


class RAGService:
    def __init__(self, embed_model, data, persist_directory = 'chroma_dir'):
        self.embed_model = embed_model
        self.persist_directory = persist_directory
        self.ini_data = data
        self.db = None

    def load_db(self):
        if os.path.exists(self.persist_directory) and os.listdir(self.persist_directory):
            print("[RAGService] Loading existing ChromaDB from disk...")
            self.db = Chroma(
                persist_directory=self.persist_directory,
                embedding_function=self.embed_model
            )
        else:
            print("[RAGService] No existing DB found at path: ", self.persist_directory)
            self.db = None

    def create_from_data(
        self,
        metadata_cols,
        txt_col,
        rag_separators=["\n\n", "\n", ".", " ", ""],
        prefix_document='search_document: ',
        chunk_size=300,
        chunk_overlap=25
    ):
        all_cols = [txt_col] + metadata_cols
        data = self.ini_data[all_cols] # !!!! no such cols

        print("[RAGService] Creating new ChromaDB...")
        splitter = RecursiveCharacterTextSplitter(
            chunk_size=chunk_size,
            chunk_overlap=chunk_overlap,
            separators=rag_separators
        )

        docs = []
        for row in data.values:
            text = row[0]
            metadata_vals = [row[data.columns.get_loc(col)] for col in metadata_cols]
            chs = splitter.split_text(text)
            for ch in chs:
                doc = Document(
                    page_content=f"{prefix_document}{ch.strip()}",
                    metadata=dict(zip(metadata_cols, metadata_vals))
                )
                docs.append(doc)

        self.db = Chroma.from_documents(
            documents=docs,
            embedding=self.embed_model,
            persist_directory=self.persist_directory
        )
        # self.db.persist() # Since Chroma 0.4.x the manual persistence method is no longer supported as docs are automatically persisted.
        print("[RAGService] New DB created and saved.")

    def search(self, query, prefix_query='search_query: ', method="similarity", k=5, filters=None):
        if self.db is None:
            raise ValueError("Database is not loaded. Please create or load a database first.")

        final_query = f'{prefix_query}{query}'
        if method == "similarity":
            if filters:
                results = self.db.similarity_search(final_query, k=k, filter=filters)
            else:
                results = self.db.similarity_search(final_query, k=k)
        elif method == "marginal":
            if filters:
                results = self.db.max_marginal_relevance_search(final_query, k=k, filter=filters)
            else:
                results = self.db.max_marginal_relevance_search(final_query, k=k)
        else:
            raise ValueError(f"Unknown search method: {method}")

        return results


class ContextConstructor:
    def __init__(
        self,
        data,
        authors_col,
        poems_col,
        txt_col,
        rag_svc,
        max_tokens=2000,
        chunk_overlap=0,
        separators=["\n\n", "\n", ".", " ", ""]
        ):
        self.data_ini = data
        self.authors_col = authors_col
        self.poems_col = poems_col
        self.txt_col = txt_col
        self.rag_svc = rag_svc
        self.max_tokens = max_tokens
        self.splitter = RecursiveCharacterTextSplitter(
            chunk_size=max_tokens * 4,  # ~4 chars per token
            chunk_overlap=chunk_overlap,
            separators=separators
        )
        self.data_ini['norm_author'] = self.data_ini[authors_col].apply(lambda x: x if pd.isnull(x) else self._normalize_author(x))
        self.data_ini['norm_poem'] = self.data_ini[poems_col].apply(lambda x: x if pd.isnull(x) else self._normalize_poem(x))

    def _normalize_author(self, name):
        return name.replace('ё', 'е').lower().strip()

    def _normalize_poem(self, name):
        return re.sub(r'[^а-яА-Яa-zA-Z0-9 ]', '', name).lower().strip()

    def _get_full_texts(self, authors, poems):
        results = []
        norm_authors = [self._normalize_author(a) for a in authors]
        norm_poems = [self._normalize_poem(p) for p in poems]

        res = []
        for a, p in zip(norm_authors, norm_poems):
          all_poem = self.data_ini.loc[(self.data_ini.norm_author == a) & (self.data_ini.norm_poem == p), self.txt_col].values#[0]
          if all_poem.shape[0] > 0:
            res.append(f"{a} '{p}':")
            res.append(all_poem[0])

        return '\n\n'.join(res)

    def _truncate_texts(self, texts):
        chunks = self.splitter.split_text(texts)
        return chunks[0] if chunks else texts

    def prepare_context(self, query, response, rag_method="similarity", k=5):
      filters = {}
      if response['is_direct'] == 1:
        texts = self._get_full_texts(response['authors'], response['poems'])
        context = self._truncate_texts(texts) if texts else ""
      else:
        if response.get('keywords'):
          rag_query = ', '.join(response['keywords'])
        else:
          rag_query = query

        if response.get("authors"):
            filters["author"] = {"$in": response["authors"]}
        if response.get("poems"):
            filters["name"] = {"$in": response["poems"]}

        results = self.rag_svc.search(rag_query, method=rag_method, k=k, filters=filters)
        if results:
          context = []
          for doc in results:
            context.append(f"{doc.metadata[self.authors_col] if doc.metadata.get(self.authors_col) else ''} '{doc.metadata[self.poems_col] if doc.metadata.get(self.poems_col) else ''}':")
            context.append(doc.page_content)
          context = "\n\n".join(context)
        else:
          context = ""

      return context

In [7]:
data = pd.read_csv(DATA_PATH)
authors_rename = {'К. Р.': 'К. Р. (Константин Романов)', 'Тэффи': 'Надежда Тэффи'}

data = data.loc[~data.text.duplicated()]
data.author = data.author.apply(lambda x: authors_rename[x] if x in authors_rename.keys() else x)
data.to_csv('russianPoetryWithTheme_deduped.csv')

In [7]:
data.shape

(16411, 9)

In [6]:
quant_config = BitsAndBytesConfig(load_in_8bit=True)
tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME, trust_remote_code=True)
model = AutoModelForCausalLM.from_pretrained(
    MODEL_NAME,
    quantization_config=quant_config,
    device_map="auto",
    trust_remote_code=True
)
model.eval()

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/4.90k [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/2.78M [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/1.67M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/11.4M [00:00<?, ?B/s]

...

model-00001-of-00004.safetensors:   0%|          | 0.00/4.87G [00:00<?, ?B/s]

model-00003-of-00004.safetensors:   0%|          | 0.00/4.33G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/242 [00:00<?, ?B/s]

Qwen2ForCausalLM(
  (model): Qwen2Model(
    (embed_tokens): Embedding(151665, 3584, padding_idx=151643)
    (layers): ModuleList(
      (0-27): 28 x Qwen2DecoderLayer(
        (self_attn): Qwen2Attention(
          (q_proj): Linear8bitLt(in_features=3584, out_features=3584, bias=True)
          (k_proj): Linear8bitLt(in_features=3584, out_features=512, bias=True)
          (v_proj): Linear8bitLt(in_features=3584, out_features=512, bias=True)
          (o_proj): Linear8bitLt(in_features=3584, out_features=3584, bias=False)
        )
        (mlp): Qwen2MLP(
          (gate_proj): Linear8bitLt(in_features=3584, out_features=18944, bias=False)
          (up_proj): Linear8bitLt(in_features=3584, out_features=18944, bias=False)
          (down_proj): Linear8bitLt(in_features=18944, out_features=3584, bias=False)
          (act_fn): SiLU()
        )
        (input_layernorm): Qwen2RMSNorm((3584,), eps=1e-06)
        (post_attention_layernorm): Qwen2RMSNorm((3584,), eps=1e-06)
      )
    )


In [5]:
embed_model = HuggingFaceEmbeddings(model_name=EMBED_MODEL_NAME)

/tmp/ipykernel_117402/585144737.py:1: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEmbeddings``.
  embed_model = HuggingFaceEmbeddings(model_name=EMBED_MODEL_NAME)
Default prompt name is set to 'Classification'. This prompt will be applied to all `encode()` calls, except if `encode()` is called with `prompt` or `prompt_name` parameters.


In [8]:
preprocessor = Preprocessor(
    model=model,
    tokenizer=tokenizer,
    data=data,
    authors_col=AUTHORS_COL,
    poems_col=POEMS_COL
)

In [10]:
from datetime import datetime

print(datetime.now())

2025-05-09 04:06:04.567765


In [8]:
print(datetime.now())
rag_svc = RAGService(embed_model, data, persist_directory = 'chroma_dir2')
# rag_svc.create_from_data(
#     metadata_cols = RAG_METADATA_COLS,
#     txt_col = RAG_TXT_COL
# )
rag_svc.load_db()
print(datetime.now())

2025-05-09 04:46:42.667844
[RAGService] Loading existing ChromaDB from disk...
2025-05-09 04:46:42.793328


/tmp/ipykernel_117402/2379853030.py:165: LangChainDeprecationWarning: The class `Chroma` was deprecated in LangChain 0.2.9 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-chroma package and should be used instead. To use it run `pip install -U :class:`~langchain-chroma` and import as `from :class:`~langchain_chroma import Chroma``.
  self.db = Chroma(


In [9]:
rag_svc.search('Осень дождь')

[Document(metadata={'name': 'Песня (С паровозами и туманами...)', 'date_to': 1955.0, 'author': 'Геннадий Шпаликов'}, page_content='search_document: В город осени и дождей.'),
 Document(metadata={'name': 'Когда вы долго слушаете споры...', 'author': 'Самуил Маршак'}, page_content='search_document: С дождем, который скоро прошумит.'),
 Document(metadata={'author': 'Юргис Балтрушайтис', 'name': 'Передо мной все тот же шум глухой...'}, page_content='search_document: Осенней ночи и тумана.'),
 Document(metadata={'name': 'Мечтания', 'date_to': 1855.0, 'author': 'Аполлон Майков'}, page_content='search_document: Пусть пасмурный октябрь осенней дышит стужей,\r\nПусть сеет мелкий дождь или порою град\r\nВ окошки звякает, рябит и пенит лужи,\r\nПусть сосны черные, качаяся, шумят,\r\nИ даже без борьбы, покорно, незаметно,\r\nСдает угрюмый день, больной и бесприветный,\r\nПрироду грустную ночной холодной мгле,—'),
 Document(metadata={'date_to': 1828.0, 'author': 'Алексей Кольцов', 'name': 'Осень'},

In [10]:
cntx_svc = ContextConstructor(
        data=data,
        authors_col=AUTHORS_COL,
        poems_col=POEMS_COL,
        txt_col=TXT_COL,
        rag_svc=rag_svc,
)

In [11]:
test_data = [
    {
        "query": "Какие стихи написал Александр Сергеевич Пушкин про любовь?",
        "expected_author": "Александр Сергеевич Пушкин",
        "expected_title": None
    },
    {
        "query": "Анализ стихотворения 'Узник' Пушкина.",
        "expected_author": "Пушкин",
        "expected_title": "Узник"
    },
    {
        "query": "Стихи о природе от Сергея Есенина.",
        "expected_author": "Сергей Есенин",
        "expected_title": None
    },
    {
        "query": "О чем рассказывает стихотворение Лермонтова 'Парус'?",
        "expected_author": "Лермонтов",
        "expected_title": "Парус"
    },
    {
        "query": "Сравните 'Бородино' Лермонтова и 'Полтаву' Пушкина.",
        "expected_author": "Лермонтов, Пушкин",
        "expected_title": "Бородино, Полтава"
    },
    {
        "query": "Какие мотивы в стихах Марины Цветаевой?",
        "expected_author": "Марина Цветаева",
        "expected_title": None
    },
    {
        "query": "Проанализируйте стихотворение 'Нивы сжаты, рощи голы' Есенина.",
        "expected_author": "Есенин",
        "expected_title": "Нивы сжаты, рощи голы"
    },
    {
        "query": "Темы осени в поэзии Блока и Ахматовой.",
        "expected_author": "Блок, Ахматова",
        "expected_title": None
    },
    {
        "query": "Смысл стихотворения \"Я помню чудное мгновенье\" Пушкина.",
        "expected_author": "Пушкин",
        "expected_title": "Я помню чудное мгновенье"
    },
    {
        "query": "Что объединяет стихи 'К Чаадаеву' Пушкина и 'Когда волнуется желтеющая нива' Тютчева?",
        "expected_author": "Пушкин, Тютчев",
        "expected_title": "К Чаадаеву, Когда волнуется желтеющая нива"
    },
    {
        'query': 'Организации объединенных наций'
    },
    {
        'query': 'О чем Бородино Лермонтова?'
    },
    {
        'query': 'О чем бородино Лермонтова?'
    },
    {
        'query': 'О чем лермонтов писал в бородино?'
    },
    {
        'query': 'проанализируй облако в штанах?'
    },
    {
        'query': 'проанализируй стихотворение маяковского облако в штанах?'
    },
    {
        'query': "проанализируй стихотворение маяковского 'Сергею Есенину'?"
    },
    {
        'query': "проанализируй стихотворение маяковского сергею есенину?"
    },
    {
        'query': "проанализируй стихотворение сергею есенину?"
    },
    {
        'query': "проанализируй 'Сергею Есенину'?"
    },
]

In [7]:
# gg = USER_PROMPT_MAIN.format(context='d1', query='d1')
# print(gg)

In [6]:
# print(gg.format(query='d2'))

In [13]:
dct_answers = {}
# for td in tqdm(test_data[:1]):
# for td in tqdm(test_data[1:2]):
for td in tqdm(test_data):
  query = td['query']
  ner = preprocessor.get_query_ner(query, SYSTEM_PROMPT, USER_PROMPT_NER)
  if ner['is_direct'] == 0:
    rewritten = preprocessor.get_query_rewrite(query, SYSTEM_PROMPT, USER_PROMPT_REWRITING)
    ner['keywords'] = rewritten
  context = cntx_svc.prepare_context(query, ner)
  context = context.replace('search_document: ', '')
  final_answ = preprocessor.generate(query, SYSTEM_PROMPT, USER_PROMPT_MAIN.format(context=context, query=query), max_new_tokens=400)
  print(final_answ)
  dct_answers[query] = final_answ

  5%|▌         | 1/20 [01:13<23:25, 73.96s/it]

system
Ты виртуальный ассистент. Твоя задача - помогать людям с поэзией, отвечая на их запросы.
user

Ты — виртуальный ассистент, помогающий людям находить и интерпретировать русскую поэзию.
Ниже приведены отрывки стихотворений, которые могут быть релевантны запросу пользователя. Используй их как источник для вдохновения или цитирования в ответе.
Не выдумывай новых стихов и не приписывай авторам лишнего — работай только с тем, что дано в контексте.
Будь образным, но точным. Если пользователь просит «подборку» — выбери релевантные строки. Если просит «написать стихи» — сочини, вдохновляясь контекстом.

Контекст:
Александр Пушкин 'Я помню чудное мгновенье...':

И сердце бьется в упоенье,
И для него воскресли вновь
И божество, и вдохновенье,
И жизнь, и слезы, и любовь.

Александр Пушкин 'К ***':

Пройдет и скроется?.. Ужель не можно мне,
Любуясь девою в печальном сладострастье,
Глазами следовать за ней и в тишине
Благословлять ее на радость и на счастье,
И сердцем ей желать все блага жизн

 10%|█         | 2/20 [02:13<19:42, 65.68s/it]

system
Ты виртуальный ассистент. Твоя задача - помогать людям с поэзией, отвечая на их запросы.
user

Ты — виртуальный ассистент, помогающий людям находить и интерпретировать русскую поэзию.
Ниже приведены отрывки стихотворений, которые могут быть релевантны запросу пользователя. Используй их как источник для вдохновения или цитирования в ответе.
Не выдумывай новых стихов и не приписывай авторам лишнего — работай только с тем, что дано в контексте.
Будь образным, но точным. Если пользователь просит «подборку» — выбери релевантные строки. Если просит «написать стихи» — сочини, вдохновляясь контекстом.

Контекст:
александр пушкин 'узник':

Сижу за решеткой в темнице сырой.
Вскормленный в неволе орел молодой,
Мой грустный товарищ, махая крылом,
Кровавую пищу клюет под окном,
Клюет, и бросает, и смотрит в окно,
Как будто со мною задумал одно;
Зовет меня взглядом и криком своим
И вымолвить хочет: "Давай улетим!
Мы вольные птицы; пора, брат, пора!
Туда, где за тучей белеет гора,
Туда, где си

 15%|█▌        | 3/20 [03:17<18:22, 64.83s/it]

...

system
Ты виртуальный ассистент. Твоя задача - помогать людям с поэзией, отвечая на их запросы.
user

Ты — виртуальный ассистент, помогающий людям находить и интерпретировать русскую поэзию.
Ниже приведены отрывки стихотворений, которые могут быть релевантны запросу пользователя. Используй их как источник для вдохновения или цитирования в ответе.
Не выдумывай новых стихов и не приписывай авторам лишнего — работай только с тем, что дано в контексте.
Будь образным, но точным. Если пользователь просит «подборку» — выбери релевантные строки. Если просит «написать стихи» — сочини, вдохновляясь контекстом.

Контекст:
Владимир Маяковский 'Канцелярские привычки':

до гор,
где грозы
гремят,
грома потрясав,-
везде
отрывки стихов и прозы,
фамилии
и адреса.
"Здесь были Соня и Ваня Хайлов.
Семейство ело и отдыхало".
"Коля и Зина
соединили души".
Стрела
и сердце
в виде груши.
"Пролетарии всех стран, соединяйтесь!
Комсомолец Петр Парулайтис".

Владимир Маяковский 'Прощание (Обыкновенно мы говорим...)

 95%|█████████▌| 19/20 [21:20<01:15, 75.14s/it]

system
Ты виртуальный ассистент. Твоя задача - помогать людям с поэзией, отвечая на их запросы.
user

Ты — виртуальный ассистент, помогающий людям находить и интерпретировать русскую поэзию.
Ниже приведены отрывки стихотворений, которые могут быть релевантны запросу пользователя. Используй их как источник для вдохновения или цитирования в ответе.
Не выдумывай новых стихов и не приписывай авторам лишнего — работай только с тем, что дано в контексте.
Будь образным, но точным. Если пользователь просит «подборку» — выбери релевантные строки. Если просит «написать стихи» — сочини, вдохновляясь контекстом.

Контекст:
Сергей Есенин 'Письмо к женщине':

Простите мне...
Я знаю: вы не та -
Живете вы
С серьезным, умным мужем;
Что не нужна вам наша маета,
И сам я вам
Ни капельки не нужен.
Живите так,
Как вас ведет звезда,
Под кущей обновленной сени.
С приветствием,
Вас помнящий всегда
Знакомый ваш
Сергей Есенин.

Сергей Есенин 'Сохнет стаявшая глина...':

Сохнет стаявшая глина,
На сугорьях гниль о

100%|██████████| 20/20 [22:31<00:00, 67.57s/it]

system
Ты виртуальный ассистент. Твоя задача - помогать людям с поэзией, отвечая на их запросы.
user

Ты — виртуальный ассистент, помогающий людям находить и интерпретировать русскую поэзию.
Ниже приведены отрывки стихотворений, которые могут быть релевантны запросу пользователя. Используй их как источник для вдохновения или цитирования в ответе.
Не выдумывай новых стихов и не приписывай авторам лишнего — работай только с тем, что дано в контексте.
Будь образным, но точным. Если пользователь просит «подборку» — выбери релевантные строки. Если просит «написать стихи» — сочини, вдохновляясь контекстом.

Контекст:
Сергей Есенин 'Сохнет стаявшая глина...':

Сохнет стаявшая глина,
На сугорьях гниль опенок.
Пляшет ветер по равнинам,
Рыжий ласковый осленок.
Пахнет вербой и смолою,
Синь то дремлет, то вздыхает.
У лесного аналоя
Воробей псалтырь читает.
Прошлогодний лист в овраге
Средь кустов, как ворох меди.
Кто-то в солнечной сермяге

Сергей Есенин 'Мелколесье. Степь и дали...':

Мелколесье. 

In [15]:
for td in tqdm(['Сгенерируй стихотворение в стиле Есенина об осени', 'Напиши стих про любовь как Пушкин']):
  # query = td['query']
  query = td
  ner = preprocessor.get_query_ner(query, SYSTEM_PROMPT, USER_PROMPT_NER)
  if ner['is_direct'] == 0:
    rewritten = preprocessor.get_query_rewrite(query, SYSTEM_PROMPT, USER_PROMPT_REWRITING)
    ner['keywords'] = rewritten
  context = cntx_svc.prepare_context(query, ner)
  context = context.replace('search_document: ', '')
  final_answ = preprocessor.generate(query, SYSTEM_PROMPT, USER_PROMPT_MAIN.format(context=context, query=query), max_new_tokens=400)
  print(final_answ)
  dct_answers[query] = final_answ

 50%|█████     | 1/2 [00:54<00:54, 54.36s/it]

system
Ты виртуальный ассистент. Твоя задача - помогать людям с поэзией, отвечая на их запросы.
user

Ты — виртуальный ассистент, помогающий людям находить и интерпретировать русскую поэзию.
Ниже приведены отрывки стихотворений, которые могут быть релевантны запросу пользователя. Используй их как источник для вдохновения или цитирования в ответе.
Не выдумывай новых стихов и не приписывай авторам лишнего — работай только с тем, что дано в контексте.
Будь образным, но точным. Если пользователь просит «подборку» — выбери релевантные строки. Если просит «написать стихи» — сочини, вдохновляясь контекстом.

Контекст:
Вероника Тушнова 'Спокойный вечер пасмурен и мглист...':

Спокойный вечер пасмурен и мглист.
Не слышно птиц среди древесных кружев.
Пустынна улица. Последний ржавый лист
в морозном воздухе легчайший ветер кружит.
Любимая осенняя пора.
На облаках - сиреневые блики,
на светлых лужицах каемка серебра,
и над землей - покой, безмерный и великий.

Семен Надсон '':

Осень, поздняя осен

100%|██████████| 2/2 [01:47<00:00, 53.76s/it]

system
Ты виртуальный ассистент. Твоя задача - помогать людям с поэзией, отвечая на их запросы.
user

Ты — виртуальный ассистент, помогающий людям находить и интерпретировать русскую поэзию.
Ниже приведены отрывки стихотворений, которые могут быть релевантны запросу пользователя. Используй их как источник для вдохновения или цитирования в ответе.
Не выдумывай новых стихов и не приписывай авторам лишнего — работай только с тем, что дано в контексте.
Будь образным, но точным. Если пользователь просит «подборку» — выбери релевантные строки. Если просит «написать стихи» — сочини, вдохновляясь контекстом.

Контекст:
Александр Пушкин 'Я помню чудное мгновенье...':

И сердце бьется в упоенье,
И для него воскресли вновь
И божество, и вдохновенье,
И жизнь, и слезы, и любовь.

Александр Пушкин 'К ***':

Пройдет и скроется?.. Ужель не можно мне,
Любуясь девою в печальном сладострастье,
Глазами следовать за ней и в тишине
Благословлять ее на радость и на счастье,
И сердцем ей желать все блага жизн

In [16]:
with open('LLMmain_responces.json', 'w') as f:
    json.dump(dct_answers, f, ensure_ascii=False)

In [ ]:
dct_answers = {}
# for td in tqdm(test_data[:1]):
# for td in tqdm(test_data[1:2]):
for td in tqdm(test_data):
  query = td['query']
  print(query)
  ner = preprocessor.get_query_ner(query, SYSTEM_PROMPT, USER_PROMPT_NER)
  if ner['is_direct'] == 0:
    rewritten = preprocessor.get_query_rewrite(query, SYSTEM_PROMPT, USER_PROMPT_REWRITING)
    ner['keywords'] = rewritten
  print(ner)
  print('-'*40)
  context = cntx_svc.prepare_context(query, ner)
  print(context)
  print('-'*80)
  dct_answers[query] = context

  0%|          | 0/20 [00:00<?, ?it/s]

Какие стихи написал Александр Сергеевич Пушкин про любовь?
{'authors': ['Александр Пушкин'], 'is_direct': 0, 'keywords': ['любовь', 'сердце', 'душа', 'нежность', 'грусть', 'встреча', 'прощание', 'звезда', 'мечта']}
----------------------------------------


  5%|▌         | 1/20 [00:21<06:47, 21.43s/it]

Александр Пушкин 'К ней (В печальной праздности...)':

search_document: И — слезы на глазах — со славою прощался!
Но вдруг, как молнии стрела,
Зажглась в увядшем сердце младость,
Душа проснулась, ожила,
Узнала вновь любви надежду, скорбь и радость.
Все снова расцвело! Я жизнью трепетал;
Природы вновь восторженный свидетель,
Живее чувствовал, свободнее дышал,

Александр Пушкин 'Под небом голубым страны своей родной...':

search_document: С таким тяжелым напряженьем,
С такою нежною, томительной тоской,
С таким безумством и мученьем!
Где муки, где любовь? Увы! в душе моей
Для бедной, легковерной тени,
Для сладкой памяти невозвратимых дней
Не нахожу ни слез, ни пени.

Александр Пушкин 'Погасло дневное светило...':

search_document: Душа кипит и замирает;
Мечта знакомая вокруг меня летает;
Я вспомнил прежних лет безумную любовь,
И все, чем я страдал, и все, что сердцу мило,
Желаний и надежд томительный обман...
Шуми, шуми, послушное ветрило,
Волнуйся подо мной, угрюмый океан.
Лети, корабль,

 10%|█         | 2/20 [00:26<03:28, 11.57s/it]

...

Владимир Маяковский 'Канцелярские привычки':

search_document: до гор,
где грозы
гремят,
грома потрясав,-
везде
отрывки стихов и прозы,
фамилии
и адреса.
"Здесь были Соня и Ваня Хайлов.
Семейство ело и отдыхало".
"Коля и Зина
соединили души".
Стрела
и сердце
в виде груши.
"Пролетарии всех стран, соединяйтесь!
Комсомолец Петр Парулайтис".

Владимир Маяковский 'Прощание (Обыкновенно мы говорим...)':

search_document: «Да нет,
вы врете —
Маяковский — поэт».—
«Ну, да,—
вмешалось двое саврасов,—
в конце
семнадцатого года
в Москве
чекой конфискован Некрасов
и весь
Маяковскому отдан.
Вы думаете —
сам он?
Сбондил до йот —
весь стих,
с запятыми,
скраден.
Достанет Некрасова
и продает —

Сергей Есенин 'Русь бесприютная':

search_document: То были б тысячи
Прекраснейших поэтов.
В них Пушкин,
Лермонтов,
Кольцов,
И наш Некрасов в них,
В них я,
В них даже Троцкий,
Ленин и Бухарин.
Не потому ль мой грустью
Веет стих,
Глядя на их
Невымытые хари.
Я знаю будущее...
Это их...
Их календарь...
И вся земная 

 95%|█████████▌| 19/20 [03:54<00:20, 20.72s/it]

Сергей Есенин 'Письмо к женщине':

search_document: Простите мне...
Я знаю: вы не та -
Живете вы
С серьезным, умным мужем;
Что не нужна вам наша маета,
И сам я вам
Ни капельки не нужен.
Живите так,
Как вас ведет звезда,
Под кущей обновленной сени.
С приветствием,
Вас помнящий всегда
Знакомый ваш
Сергей Есенин.

Сергей Есенин 'Сохнет стаявшая глина...':

search_document: Сохнет стаявшая глина,
На сугорьях гниль опенок.
Пляшет ветер по равнинам,
Рыжий ласковый осленок.
Пахнет вербой и смолою,
Синь то дремлет, то вздыхает.
У лесного аналоя
Воробей псалтырь читает.
Прошлогодний лист в овраге
Средь кустов, как ворох меди.
Кто-то в солнечной сермяге

Сергей Есенин 'Проплясал, проплакал дождь весенний...':

search_document: Проплясал, проплакал дождь весенний,
Замерла гроза.
Скучно мне с тобой, Сергей Есенин,
Подымать глаза...
Скучно слушать под небесным древом
Взмах незримых крыл:
Не разбудишь ты своим напевом
Дедовских могил!
Привязало, осаднило слово
Даль твоих времен.

Сергей Есенин 'По-о

100%|██████████| 20/20 [04:09<00:00, 12.47s/it]

Сергей Есенин 'Мелколесье. Степь и дали...':

search_document: Мелколесье. Степь и дали.
Свет луны во все концы.
Вот опять вдруг зарыдали
Разливные бубенцы.
Неприглядная дорога,
Да любимая навек,
По которой ездил много
Всякий русский человек.
Эх вы, сани! Что за сани!
Звоны мерзлые осин.
У меня отец - крестьянин,
Ну, а я - крестьянский сын.

Сергей Есенин 'Сохнет стаявшая глина...':

search_document: Сохнет стаявшая глина,
На сугорьях гниль опенок.
Пляшет ветер по равнинам,
Рыжий ласковый осленок.
Пахнет вербой и смолою,
Синь то дремлет, то вздыхает.
У лесного аналоя
Воробей псалтырь читает.
Прошлогодний лист в овраге
Средь кустов, как ворох меди.
Кто-то в солнечной сермяге

Сергей Есенин 'Запели тесаные дроги...':

search_document: Запели тесаные дроги,
Бегут равнины и кусты.
Опять часовни на дороге
И поминальные кресты.
Опять я теплой грустью болен
От овсяного ветерка.
И на известку колоколен
Невольно крестится рука.
О Русь, малиновое поле
И синь, упавшая в реку,
Люблю до радости и б

In [ ]:
with open('LLM_PreprocessRAG_responces.json', 'w') as f:
    json.dump(dct_answers, f, ensure_ascii=False)